### Output sensor locations

- Select all sensors in area of study
- Re-format in correct format for sumo / Dfrouter
- Output list/index of sensors ready for next code to generate time series

Format of detector file:

```
<detectors>
    <detectorDefinition id="<DETECTOR_ID>" lane="<LANE_ID>" pos="<POS>"/>
... further detectors ...
</detectors>
```

https://sumo.dlr.de/docs/Demand/Routes_from_Observation_Points.html#computing_detector_types

In [1]:
import osmnx as ox
import pandas as pd
from shapely.geometry import box, Polygon, MultiPolygon, Point, LineString, mapping
import geopandas as gpd
import geojson
import mysql.connector
import csv
from shapely.ops import split, snap
from sklearn.neighbors import NearestNeighbors

from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [2]:
# Get sensors
sensors = pd.read_csv('Data/midas_sensor_locations.csv')

#Get sensor distances
distance_df = pd.read_csv('Data/sensors_with_distances.csv', index_col=0)

sensors = sensors.merge(distance_df[['site ID','sp length']],left_on = 'site_ID',right_on = 'site ID', how='left').drop('site ID', axis=1)

#Read in sensors for A20 route

with open('Data/route1_a20.csv', newline='') as f:
    a20_sensors = f.read().splitlines()
print(a20_sensors)

a20_sensor = sensors[sensors['site_ID'].isin(a20_sensors)]

a20_sensor = a20_sensor.sort_values('sp length')

['5892/1', '5889/1', 'A20/7113A', 'M20/7095A', 'M20/7076A', 'M20/7018A', 'M20/6913A', 'M20/6905A', 'M20/6868A', 'M20/6645A', 'M20/6590A', 'M20/6582A', 'M20/6576A', 'M20/6572A', 'M20/6570A', 'M20/6568A1', 'M20/6552A1', 'M20/6545A', 'M20/6547A1', 'M20/6540A', 'M20/6534A', 'M20/6538A', 'M20/6531A', 'M20/6526A', 'M20/6518A', 'M20/6520A', 'M20/6523A', 'M20/6514A', 'M20/6517A', 'M20/6506A', 'M20/6511A', 'M20/6501A', 'M20/6498A', 'M20/6481A', 'M20/6484A', 'M20/6487A', 'M20/6491A', 'M20/6494A', 'M20/6477A', 'M20/6460A', 'M20/6465A', 'M20/6468A', 'M20/6472A', 'M20/6476A', 'M20/6454A', 'M20/6404A']


In [43]:
#Define line object between all sensors

point1 = Point(a20_sensor.iloc[0]['Longitude'],a20_sensor.iloc[0]['Latitude'])

line_objects = []
buffered_line_objects = []
for i,r, in a20_sensor[1:].iterrows():
    point2 = Point(r['Longitude'],r['Latitude'])
    sensor_line = LineString([point1,point2])
    line_objects.append(sensor_line)
    buffered_line_objects.append(sensor_line.buffer(0.009999))
    point1 = point2

# Create a FeatureCollection from the list of LineString objects
feature_collection = geojson.FeatureCollection([geojson.Feature(geometry=line) for line in line_objects])
# Convert the FeatureCollection to a GeoJSON string
geojson_string = geojson.dumps(feature_collection, indent=2)
    
# Convert buffered LineString objects to GeoJSON format
features = []
for buffered_line in buffered_line_objects:
    feature = geojson.Feature(geometry=mapping(buffered_line))
    features.append(feature)

# Create a FeatureCollection from the list of buffered LineString objects
feature_collection = geojson.FeatureCollection(features)

# Convert the FeatureCollection to a GeoJSON string
geojson_string = geojson.dumps(feature_collection, indent=2)
    
#Construct a polygon from line buffers

# Use unary_union to join the buffered LineString objects into a single polygon
A20_buffered_object = MultiPolygon(buffered_line_objects).buffer(0)  # Buffering with 0 effectively removes any small artifacts

In [44]:
# Extract route from OSMNX from buffer
#Get network for area just for pimary roads etc
cf = '["highway"~"motorway|motorway_link|primary|trunk"]'
G = ox.graph_from_polygon(A20_buffered_object, network_type='drive', custom_filter=cf, simplify=True)

node_attributes, edge_attributes = ox.graph_to_gdfs(G, nodes=True)

#Tidy Network
#Where there is multiple road types select 1 (at random)
road_types = []
for i in list(edge_attributes['highway']):
    if isinstance(i, str): 
        road_types.append(i)
    else:
        road_types.append(i[0])
edge_attributes['highway'] = road_types

#For edges with missing speed impute using mode
road_type_speed_limit = {}
for rtype in list(edge_attributes['highway'].value_counts().index):
    road_type_speed_limit[rtype] = edge_attributes[edge_attributes['highway'] == rtype]['maxspeed'].mode().values[0]

#Where speed missing or multuple, take from dict
speed = []
for i,r in edge_attributes.iterrows():
    #String case
    if isinstance(r['maxspeed'], str):
        speed.append(int("".join(filter(str.isdigit, r['maxspeed']))))
    #List case
    else:
        speed.append(int("".join(filter(str.isdigit, road_type_speed_limit[r['highway']]))))

edge_attributes['speed'] = speed

Each sensor described by:

- ID
- Lane
- Position along road

Process:
- Get ID for sensor
- Check how many lanes from TS data
- Expand by numer of lanes
- Adapt logic to split edges to find nodes, but get number of metres along road
    - Get proportion along road and then use to divide length
- Output in prescribed format

In [5]:
#Set up cursor object to query time-series database
host = "localhost"
user = "chris"
password = "password"
database = "midas"

# Establish a database connection
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = connection.cursor(buffered=True)

#Get Lanes per Sensor

# Get time series data for sensor
id_list = list(a20_sensor['site_ID'])
sql_query = "select distinct site_ID, lane from full_data where site_ID in {} and yr = '2022'".format(tuple(id_list))
cursor.execute(sql_query)
result = cursor.fetchall()
sensor_id_lanes = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])

print(sensor_id_lanes)

sensor_points = []
for i,r in a20_sensor.iterrows():
    sensor_points.append(Point(r['Longitude'],r['Latitude']))
    
#Compute nearest edges - needs to be recomputed each time, as structure of G fundamentally changes with each iteration
sensor_nearest_edges, distances = ox.distance.nearest_edges(G, list(a20_sensor['Longitude']), list(a20_sensor['Latitude']), interpolate=None, return_dist=True)

In [28]:
sensor_details_list = []

for sens_ind in range(len(sensor_points)):
    
    sensor_details_append = {}
    
    #Get point
    sensor_point = sensor_points[sens_ind]
    #Get ID
    sensor_id = id_list[sens_ind]
    sensor_details_append['ID'] = sensor_id
    #Append Popint
    sensor_details_append['geometry'] = sensor_points[sens_ind]
    #Get edge
    edge = G.edges[sensor_nearest_edges[sens_ind][0],sensor_nearest_edges[sens_ind][1],0]
    sensor_details_append['Edge ID'] = tuple([sensor_nearest_edges[sens_ind][0],sensor_nearest_edges[sens_ind][1]])

    #Edge Geometry
    edge_geom = edge['geometry']
    line_coords = list(edge_geom.coords)

    # Flatten the coordinates for use with NearestNeighbors
    X = [(x, y) for x, y in line_coords]
    # Fit a NearestNeighbors model
    nn_model = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X)
    # Query the model with the coordinates of the Point
    query_result = nn_model.kneighbors([[sensor_point.x, sensor_point.y]])
    # Get the index of the nearest point on the LineString
    nearest_point_index = query_result[1][0][0]
    # Extract coordinates of the nearest point on the LineString
    nearest_coordinates = line_coords[nearest_point_index]

    # Split edge geometry at nearest point
    split_line = split(edge_geom, Point(nearest_coordinates))

    #Get new geometries
    segments = [feature for feature in split_line.geoms]
    gdf_segments = gpd.GeoDataFrame(list(range(len(segments))), geometry=segments)
    gdf_segments.columns = ['index', 'geometry']

    if len(gdf_segments) > 1:
        proportion = gdf_segments.iloc[0]['geometry'].length / (gdf_segments.iloc[0]['geometry'].length + gdf_segments.iloc[1]['geometry'].length)
        metres = edge['length'] * proportion
        sensor_details_append['dist_m'] = metres
    else:
        geom_coords_list = [(x, y) for x, y in gdf_segments.iloc[0]['geometry'].coords]
        
        distances = []
        for p in geom_coords_list:
            distances.append(haversine(p[0],p[1],sensor_point.x,sensor_point.y) * 1000)
        if distances[0] < distances[-1]:
            metres = distances[0]
            sensor_details_append['dist_m'] = metres
        else:
            metres = -distances[1]
            sensor_details_append['dist_m'] = metres
            
    sensor_details_list.append(sensor_details_append)

In [29]:
sensor_details = pd.DataFrame(sensor_details_list)

### Next steps

- Get osm network and convert sumo format
- Map sumo edges to OSM edges
- Convert sensor_details IDs to Sumo format
- Output as json as appropriate

In [46]:
# Read in .net.xml file

import xml.etree.ElementTree as ET

In [52]:
tree = ET.parse('Data/Networks/route1/route1.net.xml')
root = tree.getroot()

In [55]:
print(root.tag)
print(root.attrib)

net
{'version': '1.9', 'junctionCornerDetail': '5', 'limitTurnSpeed': '5.50', '{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation': 'http://sumo.dlr.de/xsd/net_file.xsd'}


In [64]:
roads_list = []
lanes_list = []

for child in root:
    if child.tag == 'edge':
        roads_list.append(child.attrib)
        for lane in child:
            lanes_list.append(lane.attrib)

In [65]:
roads = pd.DataFrame(roads_list)
lanes = pd.DataFrame(lanes_list)

In [66]:
roads

,id,function,from,to,priority,type,spreadType,shape
0,:10302792229_0,internal,NaN,NaN,NaN,NaN,NaN,NaN
1,:10302792229_2,internal,NaN,NaN,NaN,NaN,NaN,NaN
2,:10303405999_0,internal,NaN,NaN,NaN,NaN,NaN,NaN
3,:10303405999_2,internal,NaN,NaN,NaN,NaN,NaN,NaN
4,:10303406000_0,internal,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2194,95,NaN,26006829,26006852,13,highway.trunk,center,NaN
2195,96,NaN,26006849,26006818,13,highway.trunk,center,NaN
2196,97,NaN,26006849,26006852,13,highway.trunk,center,NaN
2197,98,NaN,26006852,26006821,13,highway.trunk,center,NaN


In [101]:
lanes

,id,index,disallow,speed,length,shape,allow,acceleration
0,:10302792229_0_0,0,pedestrian bicycle tram rail_urban rail rail_e...,16.91,7.39,"70571.86,3227.00 70570.61,3225.43 70569.71,322...",NaN,NaN
1,:10302792229_0_1,1,pedestrian bicycle tram rail_urban rail rail_e...,17.88,7.39,"70574.33,3224.96 70572.83,3223.10 70571.74,322...",NaN,NaN
2,:10302792229_2_0,0,pedestrian bicycle tram rail_urban rail rail_e...,7.31,8.20,"70574.46,3220.82 70572.43,3222.22 70570.50,322...",NaN,NaN
3,:10302792229_2_1,1,pedestrian bicycle tram rail_urban rail rail_e...,5.56,4.42,"70572.41,3218.36 70571.22,3219.23 70570.16,321...",NaN,NaN
4,:10303405999_0_0,0,pedestrian bicycle tram rail_urban rail rail_e...,5.89,6.54,"39463.91,8495.16 39461.79,8493.77 39460.38,849...",NaN,NaN
...,...,...,...,...,...,...,...,...
3522,97_1,1,pedestrian bicycle tram rail_urban rail rail_e...,27.78,443.39,"17635.31,23448.97 18055.77,23308.22",NaN,NaN
3523,98_0,0,pedestrian bicycle tram rail_urban rail rail_e...,17.88,5.63,"18063.25,23306.71 18067.54,23310.36",NaN,NaN
3524,98_1,1,pedestrian bicycle tram rail_urban rail rail_e...,17.88,5.63,"18061.18,23309.15 18065.47,23312.80",NaN,NaN
3525,99_0,0,pedestrian bicycle tram rail_urban rail rail_e...,26.82,104.77,"18240.52,23628.58 18232.22,23733.02",NaN,NaN


In [68]:
lane_example = lanes.iloc[0]['shape']

In [69]:
lane_example

'70571.86,3227.00 70570.61,3225.43 70569.71,3224.37 70568.65,3223.50 70566.93,3222.50'

In [86]:
import utm

In [96]:
offset = [310981.68,5660999.41]

In [99]:
easting = 70571.86 + offset[0]
northing = 3227.00 + offset[1]

In [100]:
utm.to_latlon(easting,northing,zone_number=31,northern=True)

(51.11725803981177, 1.3076765396048835)

In [ ]:
# Process

# Convert all shapes points in lanes to lat/lon
# Associate each to a lane_id
# knn on sensors to lane points
# for nearest points get lane
# get distance to start and end point of shape
# use this to get proportion of road and then divide length by this amount
# output and observe

In [129]:
network_points = []
ind_to_lane = {}
point_ind = 0 

for i,r, in lanes.iterrows():
    for coord in r['shape'].split():
        easting = float(coord.split(',')[0]) + offset[0]
        northing = float(coord.split(',')[1]) + offset[1]

        lon = utm.to_latlon(easting,northing,zone_number=31,northern=True)[0]
        lat = utm.to_latlon(easting,northing,zone_number=31,northern=True)[1]

        network_points.append(Point(lon,lat))
        ind_to_lane[point_ind] = r['id']

        point_ind += 1